10- A partir de la plataforma online (e-shop) de los países en los que opera,
Nintendo tiene información de ventas de videojuegos diarias digitales por país
en el siguiente RDD: (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias).
Por otro lado se tienen otro RDD que tiene información de todos los videojuegos
que se venden en su plataforma con el siguiente formato (id_videojuego, titulo, rating_pegi, rating_esbr).
Tener en cuenta que un mismo videojuego se puede vender en distintos países y
esos nos permitirá obtener métricas a nivel global. Con esta información escribir
un programa en pySpark que permita:

a) Obtener el videojuego con más ventas digitales
globales (es decir en todos los países) en un RDD con el siguiente formato:
(id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales

In [7]:
import pyspark as ps
import pandas as pd

# create the Spark Session
spark = ps.sql.SparkSession.builder.getOrCreate()
# create the Spark Context
sc = spark.sparkContext

aux = sc.textFile('/home/marcos/PycharmProjects/PracticaSpark/Guia/Ej10a.csv')
dailySales = aux.map(lambda x: x.split(","))
dailySales = dailySales.map(lambda x: (pd.to_numeric(x[0], errors='coerce'), pd.to_numeric(x[1], errors='coerce'),
                         pd.to_datetime(x[2], 'coerce', format='%Y-%m-%d'),
                         pd.to_numeric(x[3], errors='coerce'), pd.to_numeric(x[4], errors='coerce'))).cache()

aux = sc.textFile('/home/marcos/PycharmProjects/PracticaSpark/Guia/Ej10b.csv')
gamesInfo = aux.map(lambda x: x.split(","))
gamesInfo = gamesInfo.map(lambda x: (pd.to_numeric(x[0], errors='coerce'), x[1],
                         pd.to_numeric(x[2], 'coerce'), pd.to_numeric(x[3], errors='coerce'))).cache()

salesByGame = dailySales.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x + y)

bestSeller = gamesInfo.map(lambda x: (x[0], x[1])).join(salesByGame).map(lambda x: (x[0], x[1][0], x[1][1])) \
        .reduce(lambda x,y: x if (x[2] > y[2]) else y)

bestSeller

(10, 'hkdmtoyh', 115)

b) Para el videojuego con mas ventas, obtener cual es el país para el cual se
registra una mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias)

In [8]:
dailySales.filter(lambda x: x[0] == bestSeller[0]).map(lambda x: (x[1], (x[3], x[4]))) \
        .reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1])).mapValues(lambda x: x[1]/x[0]) \
        .reduce(lambda x,y: x if (x[1] > y[1]) else y)[0]

9